# Загружаем таблицу с проекта РеформаЖКХ

In [1]:
import pandas as pd

In [11]:
# https://xn--80aq1a.xn--p1aee.xn--p1ai/opendata?gid=2211647&cids=house_management&page=1&pageSize=12
#https://аис.фрт.рф/opendata/export/155
url = {
        'krasnoyarsk_building' : 'https://xn--80aq1a.xn--p1aee.xn--p1ai/opendata/export/155',
        'krasnoyarsk_uo' : 'https://xn--80aq1a.xn--p1aee.xn--p1ai/opendata/export/70'
        }
storage_options = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}

In [12]:

building_gkh = pd.read_csv(url['krasnoyarsk_building'], storage_options=storage_options, encoding='utf-8', sep=';', compression='zip', engine='python')
uo_gkh = pd.read_csv(url['krasnoyarsk_uo'], storage_options=storage_options, encoding='utf-8', sep=';', compression='zip', engine='python')

In [ ]:
building_gkh = pd.read_csv('../data/export-reestrmkd-24-20230701.zip', encoding='utf-8', sep=';', compression='zip')
uo_gkh = pd.read_csv('../data/export-reestruo-24-20230701.zip', encoding='utf-8', sep=';', compression='zip')

## Выделяем таблицы для отдельных городов

In [13]:
kras_building_gkh = building_gkh[building_gkh.formalname_city == 'Красноярск']
achinsk_building_gkh = building_gkh[building_gkh.formalname_city == 'Ачинск']
kansk_building_gkh = building_gkh[building_gkh.formalname_city == 'Канск']
zelenogorsk_building_gkh = building_gkh[building_gkh.formalname_city == 'Зеленогорск']
cities = {
    'Krasnoyarsk' : kras_building_gkh,
    'Achinsk' : achinsk_building_gkh,
    'Kansk' : kansk_building_gkh,
    'Zelenogorsk' : zelenogorsk_building_gkh
}

In [14]:
for city in cities:
    cities[city].to_csv('../data/gkh_base_' + city + '.csv')
    dup = pd.DataFrame(cities[city].duplicated(keep=False, subset=['houseguid']), columns=['dup'])
    dup.reset_index(inplace=True)
    cities[city].reset_index(inplace=True)
    dup_city_dup = cities[city].merge(dup, how='left', left_on='index', right_on='index')
    dup_city_dup[dup_city_dup.dup == True].sort_values(by=['houseguid','id'], ascending=[True, True]).to_csv('../data/gkh_duplicates_' + city + '.csv')

## Удаляем дубликаты по маске

Предварительно надо вручную открыть файл duplicates, добавить столбец `delete` и вписать значения `True` либо `False` - удалить, либо оставить запись.

In [15]:
duplicates_Krasnoyarsk = pd.read_csv('../data/gkh_duplicates_Krasnoyarsk_delete_mask.csv', encoding='utf-8', sep='\t')

gkh_clear_Krasnoyarsk = kras_building_gkh.merge(duplicates_Krasnoyarsk[['id','delete']], how='left', left_on='id', right_on='id')
gkh_clear_Krasnoyarsk.fillna(value={'delete':False}, inplace=True)
gkh_clear_Krasnoyarsk = gkh_clear_Krasnoyarsk[gkh_clear_Krasnoyarsk.delete == False]
gkh_clear_Krasnoyarsk.drop(columns='delete', inplace=True)
gkh_clear_Krasnoyarsk.to_csv('../data/gkh_clear_Krasnoyarsk.csv')

In [16]:
duplicates_achinsk = pd.read_csv('../data/gkh_duplicates_Achinsk_delete_mask.csv', encoding='utf-8', sep=',')

gkh_clear_Achinsk = achinsk_building_gkh.merge(duplicates_achinsk[['id','delete']], how='left', left_on='id', right_on='id')
gkh_clear_Achinsk.fillna(value={'delete':False}, inplace=True)
gkh_clear_Achinsk = gkh_clear_Achinsk[gkh_clear_Achinsk.delete == False]
gkh_clear_Achinsk.drop(columns='delete', inplace=True)
gkh_clear_Achinsk.to_csv('../data/gkh_clear_Achinsk.csv')

In [17]:
duplicates_kansk = pd.read_csv('../data/gkh_duplicates_Achinsk_delete_mask.csv', encoding='utf-8', sep=',')

gkh_clear_Kansk = kansk_building_gkh.merge(duplicates_kansk[['id','delete']], how='left', left_on='id', right_on='id')
gkh_clear_Kansk.fillna(value={'delete':False}, inplace=True)
gkh_clear_Kansk = gkh_clear_Kansk[gkh_clear_Kansk.delete == False]
gkh_clear_Kansk.drop(columns='delete', inplace=True)
gkh_clear_Kansk.to_csv('../data/gkh_clear_Kansk.csv')

In [18]:
duplicates_zelenogorsk = pd.read_csv('../data/gkh_duplicates_Zelenogorsk_delete_mask.csv', encoding='utf-8', sep=',')

gkh_clear_Zelenogorsk =zelenogorsk_building_gkh.merge(duplicates_zelenogorsk[['id','delete']], how='left', left_on='id', right_on='id')
gkh_clear_Zelenogorsk.fillna(value={'delete':False}, inplace=True)
gkh_clear_Zelenogorsk = gkh_clear_Zelenogorsk[gkh_clear_Zelenogorsk.delete == False]
gkh_clear_Zelenogorsk.drop(columns='delete', inplace=True)
gkh_clear_Zelenogorsk.to_csv('../data/gkh_clear_Zelenogorsk.csv')

# Сравниваем с ранее геокодированными записями

In [26]:
geocoded_building_gkh = pd.read_csv('../data/geocoded_zelenogorsk.csv', encoding='utf-8', sep=',')
clear_building_gkh = pd.read_csv('../data/gkh_clear_Zelenogorsk.csv', encoding='utf-8', sep=',')

In [27]:
merged_building_gkh = clear_building_gkh.merge(geocoded_building_gkh[['id', 'houseguid', 'lon_ya', 'lat_ya', 'addr_ya']], how='left', left_on='houseguid', right_on='houseguid')
merged_building_gkh.to_csv('../data/merged_zelenogorsk.csv')

In [28]:
merged_building_gkh[merged_building_gkh['addr_ya'].isnull()]


,Unnamed: 0,index,id_x,region_id,area_id,city_id,street_id,shortname_region,formalname_region,shortname_area,...,sewerage_type,sewerage_cesspools_volume,gas_type,ventilation_type,firefighting_type,drainage_type,id_y,lon_ya,lat_ya,addr_ya
51,51,4913,9427721,db9c4f8b-b706-40e2-b2b4-d31b98dcd3d1,NaN,38f208d6-6f29-4d30-b826-b42e8beecc67,228a666e-ccd8-42bd-a3b3-acae2e0d9611,край,Красноярский,NaN,...,Не заполнено,NaN,Не заполнено,Не заполнено,Не заполнено,Не заполнено,NaN,NaN,NaN,NaN


In [29]:
merged_building_gkh[merged_building_gkh['addr_ya'] == u'Ачинск, Красноярский край, Россия']

,Unnamed: 0,index,id_x,region_id,area_id,city_id,street_id,shortname_region,formalname_region,shortname_area,...,sewerage_type,sewerage_cesspools_volume,gas_type,ventilation_type,firefighting_type,drainage_type,id_y,lon_ya,lat_ya,addr_ya
